In [4]:
# =============================================================================
# Grazioso Salvare Animal Shelter Dashboard
# =============================================================================
# Purpose: Interactive dashboard for identifying rescue dog candidates
# Author: Sarvarbek Fazliddinov
# Course: CS 340 - Client/Server Development
# =============================================================================

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure JupyterDash for proper proxy configuration
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the CRUD Python module
from crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Database credentials for AAC user account
username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# Retrieve all documents from the database using the read method
# Sending an empty document {} requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ returns '_id' column with ObjectID type which causes DataTable to crash
# Remove this column from the dataframe
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################

# Initialize the Dash application
app = JupyterDash(__name__)

# Load and encode the Grazioso Salvare logo
image_filename = 'Grazioso_Salvare_Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Define the dashboard layout
app.layout = html.Div([
    # Header section with logo and title
    html.Div([
        html.Center([
            html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                style={'height': '100px', 'margin': '10px'}
            ),
            html.H1('Grazioso Salvare Animal Shelter Dashboard'),
            html.H4('Created by: Sarvarbek Fazliddinov - CS 340'),
        ])
    ]),
    html.Hr(),
    
    # Interactive filtering options using radio buttons
    html.Div([
        html.Label('Select Rescue Type Filter:', style={'fontWeight': 'bold', 'fontSize': '16px'}),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': ' Water Rescue', 'value': 'water'},
                {'label': ' Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': ' Disaster or Individual Tracking', 'value': 'disaster'},
                {'label': ' Reset (Show All)', 'value': 'reset'}
            ],
            value='reset',
            inline=True,
            style={'margin': '10px', 'fontSize': '14px'},
            inputStyle={'marginRight': '5px', 'marginLeft': '15px'}
        )
    ], style={'padding': '15px', 'backgroundColor': '#f8f9fa', 'borderRadius': '5px'}),
    html.Hr(),
    
    # Interactive data table
    html.Div([
        dash_table.DataTable(
            id='datatable-id',
            columns=[
                {"name": i, "id": i, "deletable": False, "selectable": True} 
                for i in df.columns
            ],
            data=df.to_dict('records'),
            # Enable pagination for better user experience
            page_size=10,
            page_action='native',
            # Enable sorting on all columns
            sort_action='native',
            sort_mode='multi',
            # Enable filtering on columns
            filter_action='native',
            # Enable row selection for map updates
            row_selectable='single',
            selected_rows=[0],
            # Style the table for better readability
            style_table={'overflowX': 'auto'},
            style_cell={
                'textAlign': 'left',
                'padding': '8px',
                'minWidth': '100px',
                'maxWidth': '200px',
                'overflow': 'hidden',
                'textOverflow': 'ellipsis'
            },
            style_header={
                'backgroundColor': '#2c3e50',
                'color': 'white',
                'fontWeight': 'bold'
            },
            style_data_conditional=[
                {
                    'if': {'row_index': 'odd'},
                    'backgroundColor': '#f8f9fa'
                }
            ]
        )
    ]),
    html.Br(),
    html.Hr(),
    
    # Charts section - side by side layout
    html.Div(
        className='row',
        style={'display': 'flex'},
        children=[
            # Pie chart for breed distribution
            html.Div(
                id='graph-id',
                className='col s12 m6',
                style={'width': '50%'}
            ),
            # Geolocation map
            html.Div(
                id='map-id',
                className='col s12 m6',
                style={'width': '50%'}
            )
        ]
    )
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to update the data table based on filter selection
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    """
    Update the data table based on the selected rescue type filter.
    
    Rescue Type Filters based on Dashboard Specifications:
    - Water Rescue: Labrador Retriever Mix, Chesapeake Bay Retriever, Newfoundland
                    Intact Female, 26-156 weeks old
    - Mountain/Wilderness: German Shepherd, Alaskan Malamute, Old English Sheepdog,
                           Siberian Husky, Rottweiler
                           Intact Male, 26-156 weeks old
    - Disaster/Individual Tracking: Doberman Pinscher, German Shepherd, Golden Retriever,
                                    Bloodhound, Rottweiler
                                    Intact Male, 20-300 weeks old
    """
    
    if filter_type == 'water':
        # Water Rescue filter query
        query = {
            'animal_type': 'Dog',
            'breed': {'$in': [
                'Labrador Retriever Mix',
                'Chesapeake Bay Retriever',
                'Newfoundland'
            ]},
            'sex_upon_outcome': 'Intact Female',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        }
    elif filter_type == 'mountain':
        # Mountain or Wilderness Rescue filter query
        query = {
            'animal_type': 'Dog',
            'breed': {'$in': [
                'German Shepherd Mix',
                'Alaskan Malamute',
                'Old English Sheepdog',
                'Siberian Husky Mix',
                'Rottweiler Mix'
            ]},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        }
    elif filter_type == 'disaster':
        # Disaster or Individual Tracking filter query
        query = {
            'animal_type': 'Dog',
            'breed': {'$in': [
                'Doberman Pinscher',
                'German Shepherd Mix',
                'Golden Retriever',
                'Bloodhound',
                'Rottweiler Mix'
            ]},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}
        }
    else:
        # Reset - return all animals
        query = {}
    
    # Query the database using the CRUD module
    filtered_df = pd.DataFrame.from_records(db.read(query))
    
    # Remove the _id column if present
    if '_id' in filtered_df.columns:
        filtered_df.drop(columns=['_id'], inplace=True)
    
    return filtered_df.to_dict('records')


# Callback to highlight selected columns in the data table
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    """Highlight selected columns with a light blue background."""
    if selected_columns is None:
        return []
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# Callback to update the pie chart based on data table content
@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data')]
)
def update_graphs(viewData):
    """
    Update the pie chart to show breed distribution based on current data table.
    """
    if viewData is None or len(viewData) == 0:
        return [html.P("No data available for chart")]
    
    # Create dataframe from current view data
    dff = pd.DataFrame.from_dict(viewData)
    
    # Check if breed column exists
    if 'breed' not in dff.columns:
        return [html.P("Breed data not available")]
    
    # Create pie chart showing breed distribution
    # Limit to top 10 breeds for readability
    breed_counts = dff['breed'].value_counts().head(10)
    
    fig = px.pie(
        names=breed_counts.index,
        values=breed_counts.values,
        title='Top 10 Breed Distribution'
    )
    
    fig.update_traces(textposition='inside', textinfo='percent+label')
    fig.update_layout(showlegend=True, legend=dict(orientation='h', y=-0.1))
    
    return [dcc.Graph(figure=fig)]


# Callback to update the geolocation map based on selected row
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, index):
    """
    Update the geolocation map to show the selected animal's location.
    """
    # Handle cases where data is not available
    if viewData is None:
        return [html.P("No data available for map")]
    
    # Create dataframe from current view data
    dff = pd.DataFrame.from_dict(viewData)
    
    # If no data, return message
    if len(dff) == 0:
        return [html.P("No data available for map")]
    
    # Determine which row to display
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]
    
    # Ensure row index is valid
    if row >= len(dff):
        row = 0
    
    # Get latitude and longitude for the selected animal
    try:
        lat = dff.iloc[row]['location_lat']
        lon = dff.iloc[row]['location_long']
        breed = dff.iloc[row]['breed'] if 'breed' in dff.columns else 'Unknown'
        name = dff.iloc[row]['name'] if 'name' in dff.columns else 'Unknown'
        
        # Handle missing or null values
        if pd.isna(lat) or pd.isna(lon):
            lat, lon = 30.75, -97.48  # Default to Austin, TX
    except:
        lat, lon = 30.75, -97.48  # Default to Austin, TX
        breed = 'Unknown'
        name = 'Unknown'
    
    # Create the map centered on Austin, TX
    return [
        dl.Map(
            style={'width': '100%', 'height': '500px'},
            center=[30.75, -97.48],
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                # Marker for the selected animal
                dl.Marker(
                    position=[lat, lon],
                    children=[
                        dl.Tooltip(breed),
                        dl.Popup([
                            html.H4("Animal Information"),
                            html.P(f"Name: {name}"),
                            html.P(f"Breed: {breed}")
                        ])
                    ]
                )
            ]
        )
    ]


# Run the application
if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on https://cannonpercent-scriptvertigo-3000.codio.io/proxy/8050/
